# Chapter 4. 문서에 꼬리표 달기  

문서 분류란 입력 문서가 어떤 범주일지 분류하는 과제입니다. 예를 들어 뉴스가 정치 기사인지, 경제 기사인지, 연예 기사인지를 가려내는 것도 문서 분류라고 말할 수 있습니다.  
이 장에서는 영화 리뷰의 감성을 분류하는 모델을 만들어 보겠습니다. 이 모델은 영화 리뷰가 긍정인지, 부정인지를 판단하는 역할을 합니다. 이 장을 차근차근 따라 하다 보면 영화 리뷰 말고도 다양한 문서 분류 모델을 만들어 볼 수 있습니다.

## <4-1 문서 분류 모델 훑어보기>  

***문서 분류(document classification)*** 란 문서가 주어졌을 때 해당 문서의 범주를 분류하는 과제입니다. 뉴스를 입력하고 정치, 경제, 연예 등 범주를 맞히거나, 영화 리뷰가 긍정/부정 등 어떤 ***극성(polarity)*** 을 가지는지 분류하는 작업이 문서 분류의 대표적인 예입니다.

이번 장에서 사용할 데이터는 네이버 영화 리뷰 말뭉치(NSMC)인데요, 우리가 만들 문서 분류 모델은 영화 리뷰 문장을 입력으로 하고 해당 문장이 속한 극성의 확률을 출력합니다. 문서 분류 모델의 출력은 확률값이므로 적당한 후처리 과정을 거쳐 긍정, 부정처럼 사람이 보기에 좋은 형태로 가공해 줍니다.  
예를 들어 다음처럼 문장의 극성을 분류하는 과제를 ***감성 분석(sentiment analysis)*** 이라고 합니다.

### 모델 구조
---
이 책에서 사용하는 문서 분류 모델은 입력 문장을 토큰화한 뒤 문장 시작과 끝을 알리는 스페셜 토큰 **CLS**와 **SEP**를 각각 원래 토큰 시퀀스 앞뒤에 붙입니다. 이를 BERT 모델에 입력하고 문장 수준의 벡터(**pooler_output**)를 뽑습니다. 이 벡터에 작은 추가 모듈을 덧붙여 모델 전체의 출력이 [해당 문장이 긍정일 확률, 해당 문장이 부정일 확률] 형태가 되도록 합니다.  

cf) 문장 수준의 벡터를 계산하는 과정은 <3-6 단어/문장을 벡터로 변환하기> 실습을 참고하세요! 

<center><그림 4-1 문서 분류></center>

<p align="center"><img src="https://i.imgur.com/5lpkDEB.png">  

<center>출처 : ratsgo's NLPBOOK</center>

### 태스크 모듈
---
**pooler_output** 벡터 뒤에 붙는 추가 모듈의 구조는 다음 그림과 같습니다.  
우선 **pooler_output** 벡터(그림에서 $x$)에 드롭아웃을 적용합니다. 드롭아웃을 적용한다는 의미는 그림에서 입력 벡터 $x$의 768개 각 요솟값 가운데 일부를 랜덤으로 0으로 바꿔 이후 계산에 포함하지 않도록 하는 것입니다.

<center><그림 4-2 문서 분류 태스크 모듈></center>

<p align="center"><img src="https://i.imgur.com/NwcI0H2.png">  

<center>출처 : ratsgo's NLPBOOK</center>

그다음 가중치 행렬을 곱해 **pooler_output**을 분류해야 할 범주 수만큼의 차원을 갖는 벡터로 변환합니다(그림에서 $h$). 만일 **pooler_output** 벡터가 768차원이고 분류 대상 범주 수가 2개(**긍정, 부정**)라면 가중치 행렬 크기는 768 $\times$ 2 가 됩니다. 여기에 소프트맥스 함수를 취하면 모델의 최종 출력이 됩니다.

이렇게 만든 모델의 최종 출력과 정답 레이블을 비교해 모델 출력이 정답 레이블과 최대한 같아지도록 태스크 모듈과 BERT 레이어를 포함한 모델 전체를 업데이트합니다.  
이를 ***파인튜닝(fine-tuning)*** 이라고 합니다.